In [1]:
import pandas as pd
import numpy as np
import json
import random

In [2]:
df_customers = pd.read_csv('./customers_and_clusters.csv')
df_clusters = pd.read_csv('./normalizados.csv')

In [3]:
def recomendar_producto_para_cluster(cluster_id: int, threshold: float = 0.01):
    
    with open('items_procesados.json', 'r', encoding='utf-16') as f:
        productos = json.load(f)

    cluster_row = df_clusters[df_clusters['CLUSTER'] == cluster_id].iloc[0]

    prob_vector = cluster_row.drop('CLUSTER').values

    ingredientes_relevantes = np.where(prob_vector >= threshold)[0]
    prob_relevantes = prob_vector[ingredientes_relevantes]

    productos_scores = []
    for producto in productos:
        vec = np.array(producto["ingredientes_vector"])
        ingredientes_producto = vec[ingredientes_relevantes]
        score = np.dot(ingredientes_producto, prob_relevantes)
        productos_scores.append((producto, score))

    productos_scores = [(p, s) for p, s in productos_scores if s > 0]

    if not productos_scores:
        return {"mensaje": "No hay productos compatibles con este cluster y threshold"}

    productos, scores = zip(*productos_scores)
    scores = np.array(scores)
    probabilidades = scores / scores.sum()

    producto_seleccionado = random.choices(productos, weights=probabilidades, k=1)[0]

    return producto_seleccionado

In [4]:
def mostrar_top5_ingredientes_y_recomendacion_un_cluster(cluster_id, threshold=0.01):
    with open("ingredientes_unicos.json", 'r', encoding='utf-16') as f:
        ingredientes_unicos = json.load(f)["ingredientes"]

    fila = df_clusters[df_clusters['CLUSTER'] == cluster_id]
    if fila.empty:
        raise ValueError(f"No se encontró el cluster con ID {cluster_id}")

    row = fila.iloc[0]
    vector_ingredientes = row.drop('CLUSTER').values

    top5_idx = np.argsort(vector_ingredientes)[-5:][::-1]
    top5_nombres = [ingredientes_unicos[i] for i in top5_idx]

    item_recomendado = recomendar_producto_para_cluster(cluster_id, threshold)['Item']

    return {
        "cluster": cluster_id,
        "ingredientes_top5": top5_nombres,
        "item_recomendado": item_recomendado
    }

In [5]:
def mostrar_top5_ingredientes_y_recomendacion():
    with open("ingredientes_unicos.json", 'r', encoding='utf-16') as f:
        ingredientes_unicos = json.load(f)["ingredientes"]

    resultados = []

    for idx, row in df_clusters.iterrows():
        cluster_id = row['CLUSTER']
        vector_ingredientes = row.drop('CLUSTER').values
        
        top5_idx = np.argsort(vector_ingredientes)[-5:][::-1]
        
        top5_nombres = [ingredientes_unicos[i] for i in top5_idx]
        
        item_recomendado = recomendar_producto_para_cluster(cluster_id)['Item']
        
        resultados.append({
            "cluster": cluster_id,
            "ingredientes_top5": top5_nombres,
            "item_recomendado": item_recomendado
        })

    return resultados

In [6]:
def agregar_recomendacion_por_usuario(user_id, threshold=0.01):
    fila_usuario = df_customers[df_customers['customer_id'] == user_id]
    
    if fila_usuario.empty:
        raise ValueError(f"El usuario con ID {user_id} no se encuentra en df_usuarios.")
    
    cluster_id = fila_usuario.iloc[0]['CLUSTER']
    
    cluster_row = df_clusters[df_clusters['CLUSTER'] == cluster_id]
    if cluster_row.empty:
        raise ValueError(f"No se encontró el cluster {cluster_id} en df_clusters.")
    
    item = recomendar_producto_para_cluster(cluster_id, threshold)
    
    return "Producto: " + item['Item'] + "\nIngredientes: " + item['ingredientes']

## Recomendar producto para un usuario por customer id

**Los resultados varían ya que se recomienda con una distribución de probabilidad**

In [30]:
# Usuario a recomendar

# Ejemplos:

# 0000b06f-82aa-44e0-9dd4-218ebeff8cfd
# 0000c481-e2b2-4b17-ae40-5d87b531c717
# 00008406-922f-4c8e-bcbd-b281670c2308
# 00008406-922f-4c8e-bcbd-b281670c2308

User_id = '00008406-922f-4c8e-bcbd-b281670c2308'

In [35]:
producto_recomendado = agregar_recomendacion_por_usuario('00008406-922f-4c8e-bcbd-b281670c2308', threshold=0.01)
print(producto_recomendado)

Producto: ITEM_M3
Ingredientes: Fire Braised Flank Steak, Green Leaf Lettuce, Corn, Black Beans, Savoy Cabbage, Feta Cheese, Red Cabbage, Tomatoes, Tortilla Strips, Red Bell Pepper, Poblano Peppers, Canola Oil, Jalapeño, Paprika, Salt, Black Pepper, Dried Rosemary


## Recomendar producto por cluster y los 5 ingredientes más representativos del cluster

**Los resultados varían ya que se recomienda con una distribución de probabilidad**

In [40]:
# Cluster a recomenda

# Ejemplos:

# 0
# 6
# 17
# 23

resultado = mostrar_top5_ingredientes_y_recomendacion_un_cluster(10)
print(f"Cluster {resultado['cluster']}: {resultado['ingredientes_top5']} -> {resultado['item_recomendado']}")

Cluster 10: ['Vanilla Extract', 'Honey', 'Salt', 'Sea Salt', 'Cane Sugar'] -> ITEM_W2


### Recomendación para todos los cluster

**Los resultados varían ya que se recomienda con una distribución de probabilidad**

In [44]:
resultados = mostrar_top5_ingredientes_y_recomendacion()
for r in resultados:
    print(f"Cluster {r['cluster']}: {r['ingredientes_top5']} -> {r['item_recomendado']}")

Cluster 0.0: ['Salt', 'Water', 'Canola Oil', 'Red Cabbage', 'Green Leaf Lettuce'] -> ITEM_C
Cluster 1.0: ['Salt', 'Canola Oil', 'Water', 'Black Pepper', 'Lemon Juice'] -> ITEM_A3
Cluster 2.0: ['Salt', 'Sea Salt', 'Honey', 'Canola Oil', 'Vegenaise'] -> ITEM_Z3
Cluster 3.0: ['Salt', 'Canola Oil', 'Cane Sugar', 'Trisodium Phosphate', 'Coffee'] -> ITEM_Q
Cluster 4.0: ['Salt', 'Water', 'Chia Seeds', 'Vanilla Extract', 'Coconut Milk'] -> ITEM_N2
Cluster 5.0: ['Salt', 'Canola Oil', 'Yellow Onion', 'Hard Boiled Eggs', 'Chicken Breast'] -> ITEM_W2
Cluster 6.0: ['Salt', 'Canola Oil', 'Red Cabbage', 'Savoy Cabbage', 'Corn'] -> ITEM_Z2
Cluster 7.0: ['Salt', 'Sea Salt', 'Water', 'High Oleic Sunflower Oil and/or Safflower Oil and/or Canola Oil', 'Stone Ground Corn'] -> ITEM_Y
Cluster 8.0: ['Vanilla Extract', 'Honey', 'Dark Chocolate', 'Sunflower Oil', 'Almond Butter'] -> ITEM_I1
Cluster 9.0: ['Salt', 'Black Pepper', 'Water', 'Red Onion', 'Vanilla Extract'] -> ITEM_M2
Cluster 10.0: ['Vanilla Extract'